In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler, StandardScaler, FunctionTransformer,PowerTransformer
import pickle
import xgboost as xgb
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.metrics import roc_curve,auc

#DATA INPUT
gst_test_in=pd.read_csv('../../GSTN_project/Test_20/Test_20/X_Test_Data_Input.csv')
gst_test_out=pd.read_csv('../../GSTN_project/Test_20/Test_20/Y_Test_Data_Target.csv')
df=pd.concat([gst_test_in,gst_test_out],axis=1)



#HANDLING MISSING DATA(either filling it or removing nan values)
df=df.dropna(subset='Column6')
df['Column0'].fillna(df['Column0'].median(),inplace=True)
cols=['Column3','Column4','Column5']
df[cols]=df[cols].fillna(df[cols].mean())
df['Column15']=df['Column15'].fillna(df['Column15'].mode()[0])
df['Column14']=df['Column14'].fillna(df['Column14'].mode()[0])


#DATA TRANSFORMATIONS

# Step 1: Remove specified columns
columns_to_remove = ['Column14', 'Column16', 'Column21', 'Column10', 'Column3', 'Column9', 'ID', 'target']  # Columns to remove
X = df.drop(columns=columns_to_remove)

# Step 2: Apply RobustScaler to specified columns
columns_robust = ['Column1', 'Column2', 'Column5', 'Column7', 'Column15']
robust_scaler = RobustScaler()
X[columns_robust] = robust_scaler.fit_transform(X[columns_robust])

# Step 3: Apply StandardScaler to the already scaled columns (Column5, Column7, Column15)
standard_scaler = StandardScaler()
X[['Column5', 'Column7', 'Column15']] = standard_scaler.fit_transform(X[['Column5', 'Column7', 'Column15']])

# Step 4: Apply Log transformation to Column5, Column7 and Column8
X[['Column5', 'Column7']] = FunctionTransformer(np.log1p, validate=True).fit_transform(X[['Column5', 'Column7']])
X['Column8'] = FunctionTransformer(np.log1p, validate=True).fit_transform(X[['Column8']])

# Step 5: Apply Yeo-Johnson transformation to Column15
X['Column15'] = PowerTransformer(method='yeo-johnson').fit_transform(X[['Column15']])

#Step 6:Capping or Winsorization (Outlier handling)(optional)
for column in ['Column5', 'Column7', 'Column8']:
    lower_bound = X[column].quantile(0.01)
    upper_bound = X[column].quantile(0.95)
    X[column] = np.clip(X[column], lower_bound, upper_bound)

X_test=X
y_test=df['target'].values

# LOADING THE MODEL FROM THE PICKLE FILE
with open('../../GSTN_project/final_model_class_adj.pkl', 'rb') as f:
    xgb_model = pickle.load(f)
with open('../../GSTN_project/final_model_oversample.pkl', 'rb') as f:
    xgb_model_oversample = pickle.load(f)
    
#PREDICTING
y_pred=xgb_model_oversample.predict(X_test)
